In [259]:
using CSV, DataFrames, JuMP, Gurobi

In [260]:
OrderList = CSV.read("OrderList.csv", DataFrame)
first(OrderList, 5)

Row,Product_ID,Plant_Code,Weight
,Int64,Int64,Float64
1,0,15,25.2027
2,1,42,73.2817
3,2,24,19.4667
4,3,96,593.702
5,4,30,26.5947


In [261]:
OrderList_matrix = Matrix(OrderList)

772×3 Matrix{Float64}:
    0.0   15.0   25.2027
    1.0   42.0   73.2817
    2.0   24.0   19.4667
    3.0   96.0  593.702
    4.0   30.0   26.5947
    5.0   12.0   11.4692
    6.0   36.0   66.3841
    7.0   25.0    9.5
    8.0   66.0   51.1902
    9.0   42.0   36.0747
   10.0    9.0  124.211
   13.0   18.0   23.7079
   14.0    9.0   18.06
    ⋮           
 1521.0    3.0   13.92
 1522.0    3.0    7.78
 1523.0    3.0    5.76
 1528.0   12.0   32.6913
 1529.0   18.0    9.2431
 1531.0    3.0   22.3942
 1534.0    3.0    1.75692
 1535.0    3.0   33.4904
 1536.0   12.0    2.9743
 1537.0  128.0  839.29
 1538.0   48.0  150.41
 1539.0   48.0  162.5

In [262]:
FreightRates = CSV.read("FreightRates.csv", DataFrame)
first(FreightRates, 5)

Row,Carrier,orig_port_cd,dest_port_cd,minm_wgh_qty,max_wgh_qty,minimum_cost,rate
,Int64,Int64,Int64,Float64,Float64,Float64,Float64
1,0,8,9,250.0,499.99,43.2272,0.7132
2,1,8,9,65.0,69.99,43.2272,0.7512
3,2,8,9,60.0,64.99,43.2272,0.7892
4,3,8,9,50.0,54.99,43.2272,0.8272
5,4,8,9,35.0,39.99,43.2272,1.0552


In [263]:
FreightRates_matrix = Matrix(FreightRates)

1540×7 Matrix{Float64}:
    0.0  8.0  9.0   250.0     499.99  43.2272  0.7132
    1.0  8.0  9.0    65.0      69.99  43.2272  0.7512
    2.0  8.0  9.0    60.0      64.99  43.2272  0.7892
    3.0  8.0  9.0    50.0      54.99  43.2272  0.8272
    4.0  8.0  9.0    35.0      39.99  43.2272  1.0552
    5.0  8.0  9.0   100.0     249.99  43.2272  0.7132
    6.0  8.0  9.0   500.0    1999.99  43.2272  0.6752
    7.0  8.0  9.0    55.0      59.99  43.2272  0.8272
    8.0  8.0  9.0    25.0      29.99  43.2272  1.2452
    9.0  8.0  9.0    30.0      34.99  43.2272  1.1312
   10.0  8.0  9.0    70.0      99.99  43.2272  0.7512
   11.0  8.0  9.0    10.0      14.99  43.2272  1.8272
   12.0  8.0  9.0     0.0       4.99  43.2272  1.8272
    ⋮                                  ⋮       
 1528.0  4.0  9.0   500.0    1999.99  31.9612  0.1816
 1529.0  4.0  9.0   100.0     249.99  31.9612  0.1816
 1530.0  2.0  9.0   500.0    1999.99  27.0384  0.124
 1531.0  2.0  9.0  2000.0  100000.0   27.0384  0.124
 1532.0  2.0

In [264]:
WhCosts = CSV.read("WhCosts.csv", DataFrame)
WhCosts = sort(WhCosts,[:WH])
first(WhCosts, 5)

Row,WH,Cost/kg
,Int64,Float64
1,1,0.566976
2,2,0.477504
3,3,0.517502
4,4,0.428503
5,5,0.488144


In [265]:
WhCosts_matrix = Matrix(WhCosts)

16×2 Matrix{Float64}:
  1.0  0.566976
  2.0  0.477504
  3.0  0.517502
  4.0  0.428503
  5.0  0.488144
  6.0  0.554088
  7.0  0.371424
  8.0  0.522857
  9.0  0.465071
 10.0  0.493582
 11.0  0.555247
 12.0  0.773132
 13.0  0.469707
 14.0  0.63433
 16.0  1.91981
 18.0  2.03625

In [266]:
WhCapacities = CSV.read("WhCapacities.csv", DataFrame)
WhCapacities = sort(WhCapacities,[:Plant_ID])
first(WhCapacities, 5)

Row,Plant_ID,Daily_Capacity
,Int64,Int64
1,1,1070000
2,2,138000
3,3,1013000
4,4,554000
5,5,385000


In [267]:
WhCapacities_matrix = Matrix(WhCapacities)

16×2 Matrix{Int64}:
  1  1070000
  2   138000
  3  1013000
  4   554000
  5   385000
  6    49000
  7   265000
  8    14000
  9   110000
 10   118000
 11   332000
 12   209000
 13   490000
 14   549000
 16   457000
 18   111000

In [268]:
ProductsPerPlant = CSV.read("ProductsPerPlant.csv", DataFrame)
ProductsPerPlant = sort(ProductsPerPlant,[:Product_ID])
first(ProductsPerPlant, 5)

Row,Product_ID,Plant_Code_1,Plant_Code_2,Plant_Code_3,Plant_Code_4,Plant_Code_5,Plant_Code_6,Plant_Code_7,Plant_Code_8,Plant_Code_9,Plant_Code_10,Plant_Code_11,Plant_Code_12,Plant_Code_13,Plant_Code_14,Plant_Code_16,Plant_Code_18
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [269]:
ProductsPerPlant_matrix = Matrix(ProductsPerPlant)


772×17 Matrix{Int64}:
    0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
    1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
    2  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0
    3  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0
    4  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
    5  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
    6  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0
    7  0  0  0  0  0  0  0  0  0  0  0  1  1  0  0  0
    8  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
    9  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
   10  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
   13  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
   14  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
    ⋮              ⋮              ⋮              ⋮  
 1521  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
 1522  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
 1523  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
 1528  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
 1529  

In [270]:
PlantPorts = CSV.read("PlantPorts.csv", DataFrame)
PlantPorts

Row,Plant_Code,Port_1,Port_2,Port_3,Port_4,Port_5,Port_6,Port_7,Port_8,Port_9,Port_10,Port_11
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,1,1,0,0,0,0,0,0,0,0,0
2,2,0,0,1,0,0,0,0,0,0,0,0
3,3,0,0,0,1,0,0,0,0,0,0,0
4,4,0,0,0,0,1,0,0,0,0,0,0
5,5,0,0,0,0,0,1,0,0,0,0,0
6,6,0,0,0,0,0,1,0,0,0,0,0
7,7,1,1,0,0,0,0,0,0,0,0,0
8,8,0,0,0,1,0,0,0,0,0,0,0
9,9,0,0,0,1,0,0,0,0,0,0,0


In [271]:
PlantPorts_matrix = Matrix(PlantPorts)

16×12 Matrix{Int64}:
  1  1  1  0  0  0  0  0  0  0  0  0
  2  0  0  1  0  0  0  0  0  0  0  0
  3  0  0  0  1  0  0  0  0  0  0  0
  4  0  0  0  0  1  0  0  0  0  0  0
  5  0  0  0  0  0  1  0  0  0  0  0
  6  0  0  0  0  0  1  0  0  0  0  0
  7  1  1  0  0  0  0  0  0  0  0  0
  8  0  0  0  1  0  0  0  0  0  0  0
  9  0  0  0  1  0  0  0  0  0  0  0
 10  1  1  0  0  0  0  0  0  0  0  0
 11  0  0  0  1  0  0  0  0  0  0  0
 12  0  0  0  1  0  0  0  0  0  0  0
 13  0  0  0  1  0  0  0  0  0  0  0
 14  0  0  0  0  0  0  1  0  0  0  0
 16  0  0  0  0  0  0  0  0  1  0  0
 18  0  0  0  0  0  0  0  0  0  0  1

In [272]:
plants = 1:16 # i
ports = 1:11 # j
types = 1:772 # 1:772, t
carriers = 1:1540 # k
M = 5000;

In [288]:
# set up model
model = Model(Gurobi.Optimizer)

# insert variables
# weight to produce at plant i of type t that are going to be transported to port j
@variable(model, x[plants, ports, types] >= 0)
# weights to transport from port j using carrier k
@variable(model, y[ports, carriers] >= 0)
# whether or not to transport from port j using carrier k
@variable(model, z[ports, carriers], Bin)

# Capacity constraint for each plant
@constraint(model, [i in plants], sum(sum(x[i, j, t] for t in types) for j in ports) <= WhCapacities_matrix[i, 2])
# Constraint for determining whether each product can be produced at plant i 
@constraint(model, [i in plants, t in types], sum(x[i, j, t] for j in ports) <= ProductsPerPlant_matrix[t, i + 1] * M)
# Demand constraint
@constraint(model, [t in types], sum(sum(x[i, j, t] for i in plants) for j in ports) >= OrderList_matrix[t])
# Plant vs. port match constraint
# @constraint(model, [j in ports], sum(y[j, k] for k in carriers) == sum((sum(x[i, t] for t in types) * PlantPorts_matrix[i, j + 1]) for i in plants))
# @constraint(model, [i in plants], sum(x[i, t] for t in types) == sum((sum(y[j, k] for k in carriers) * PlantPorts_matrix[i, j + 1]) for j in ports))
# Ensure all that are produced are transported
@constraint(model, sum(sum(sum(x[i, j, t] for i in plants) for t in types) for j in ports) == sum((sum(y[j, k] for j in ports) for k in carriers)))
# Ensure all products produced at plant i for type t to port j has arrived at port j
@constraint(model, [j in ports], sum(sum(x[i, j, t] for t in types) for i in plants) == sum(y[j, k] for k in carriers))
# Constraint for which plant can be shipped to which port
@constraint(model, [t in types, j in ports, i in plants], x[i, j, t] <= PlantPorts_matrix[i, j + 1] * M)
# Transportation constraint (min and max weight per carrier)
@constraint(model, [j in ports, k in carriers], y[j, k] <= z[j, k] * FreightRates_matrix[k, 5]) # max weight
@constraint(model, [j in ports, k in carriers], y[j, k] >= z[j, k] * FreightRates_matrix[k, 4]) # min weight
# Valid starting port for carrier constraint
for k in carriers
    for j in ports
        if FreightRates_matrix[k, 2] != j
            @constraint(model, z[j, k] == 0)
        end
    end
end

# insert objective function    
@objective(model, Min, sum(sum(sum(WhCosts_matrix[i, 2] * x[i, j, t] for t in types) for i in plants) for j in ports) + sum(sum(FreightRates_matrix[k, 6] + FreightRates_matrix[k, 7] * y[j, k] for j in ports) for k in carriers))

# solve
optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-20
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Max
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 198304 rows, 169752 columns and 931260 nonzeros
Model fingerprint: 0xd51e20ae
Variable types: 152812 continuous, 16940 integer (16940 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+05]
  Objective range  [3e-02, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 195739 rows and 166834 columns
Presolve time: 0.10s
Presolved: 2565 rows, 2918 columns, 7039 nonzeros
Variable types: 1720 continuous, 1198 integer (1198 binary)

Root relaxation: objective 5.792718e+05, 268 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

In [289]:
JuMP.value.(x)

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, 1:16
    Dimension 2, 1:11
    Dimension 3, 1:772
And data, a 16×11×772 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [290]:
produced = []
xvalues = JuMP.value.(x)

for i in plants
    sum = 0
    for j in ports
        for t in types
            sum += xvalues[i, j, t]
        end
    end
    append!(produced, sum)
end

produced

16-element Vector{Any}:
      0.0
      0.0
 395970.0
  39179.0
  16342.0
      0.0
   8488.0
    509.0
   9546.0
  48868.0
      0.0
  26040.0
  14042.0
      0.0
  27218.0
      0.0

In [291]:
JuMP.value.(y)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:11
    Dimension 2, 1:1540
And data, a 11×1540 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [292]:
transported = []
yvalues = JuMP.value.(y)

for j in ports
    sum = 0
    for k in carriers
        sum += yvalues[j, k]
    end
    append!(transported, sum)
end
transported

11-element Vector{Any}:
      0.0
  57356.0
      0.0
 446107.0
  39179.0
  16342.0
      0.0
      0.0
  27218.0
      0.0
      0.0

In [294]:
sum(xvalues) == sum(yvalues)

true

In [299]:
carriers_used = Dict()

for k in carriers
    sum = 0
    for j in ports
        sum += yvalues[j, k]
    end
    if sum != 0
        carriers_used[k] = sum
    end
end

carriers_used

Dict{Any, Any} with 39 entries:
  392 => 99.99
  867 => 249.99
  52  => 499.99
  60  => 99.99
  866 => 100000.0
  870 => 99.99
  864 => 100000.0
  53  => 32057.2
  187 => 499.99
  49  => 249.99
  41  => 99.99
  43  => 249.99
  188 => 100000.0
  190 => 100000.0
  189 => 249.99
  185 => 1999.99
  39  => 1999.99
  51  => 1999.99
  61  => 499.99
  865 => 249.99
  394 => 13492.0
  861 => 1999.99
  868 => 499.99
  869 => 1999.99
  393 => 1999.99
  ⋮   => ⋮

In [302]:
zvalues = JuMP.value.(z)
carriers_from_port = []
for j in ports
    sum = 0
    for k in carriers
        if zvalues[j, k] != 0
            sum += 1
        end
    end
    append!(carriers_from_port, sum)
end

carriers_from_port

11-element Vector{Any}:
  0
 16
 29
 29
  6
 11
  4
  5
 12
  6
  7